<a href="https://colab.research.google.com/github/toandaominh1997/Kaggle/blob/master/%20House%20Prices%3A%20Advanced%20Regression%20Techniques/House_Prices_Advanced_Regression_Techniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import pandas as pd
import numpy as np
import os

path_train = 'drive/My Drive/Datasets/House Prices: Advanced Regression Techniques/train.csv'
path_test = 'drive/My Drive/Datasets/House Prices: Advanced Regression Techniques/test.csv'
train=pd.read_csv(path_train)
test=pd.read_csv(path_test)
testID = test['Id']
target = train['SalePrice']
train = train.drop(columns=['SalePrice', 'Id'], axis=1)
test = test.drop(columns=['Id'], axis=1)

train = train.drop(train[(train['GrLivArea']>4000) & (train['SalePrice']<300000)].index)



all_data = pd.concat([train, test], axis=0)

all_data_na = (all_data.isnull().sum() / len(all_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")
all_data["LotFrontage"] = all_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')
    
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])
all_data = all_data.drop(['Utilities'], axis=1)
all_data["Functional"] = all_data["Functional"].fillna("Typ")
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])
all_data['MSSubClass'] = all_data['MSSubClass'].fillna("None")
all_data['MSSubClass'] = all_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
all_data['OverallCond'] = all_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
all_data['YrSold'] = all_data['YrSold'].astype(str)
all_data['MoSold'] = all_data['MoSold'].astype(str)

from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(all_data[c].values)) 
    all_data[c] = lbl.transform(list(all_data[c].values))

# shape        
print('Shape all_data: {}'.format(all_data.shape))
all_data['TotalSF'] = all_data['TotalBsmtSF'] + all_data['1stFlrSF'] + all_data['2ndFlrSF']
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [33]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def rmse(y_true, y_pred):
  return np.sqrt(mean_squared_error(y_true, y_pred))
def pd2np(data, dtype=np.float32):
  return np.asarray(data, dtype=dtype)

train = pd2np(train)
from sklearn.linear_model import SGDRegressor
from sklearn import svm
from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

kf = KFold(n_splits=20)
clf = SGDRegressor()
# clf = Ridge(alpha=.5)
# clf = svm.SVR()
# clf = linear_model.Lasso(alpha=0.1)
# clf = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)
bestmodel = clf
bestscore = 10000000000
for train_index, val_index in kf.split(train):
  X_train, X_val= train[train_index], train[val_index]
  y_train, y_val = target[train_index], target[val_index]
  clf.fit(X_train, y_train)
  print("Score RMSE = ", rmse(y_val, clf.predict(X_val)))
  score = rmse(y_val, clf.predict(X_val))
  if(bestscore> score):
    bestscore = score
    bestmodel = clf
  



Score RMSE =  4.676780258332672e+16
Score RMSE =  3.6679070813602184e+16
Score RMSE =  5.909105595116116e+17
Score RMSE =  2.0241403600057296e+17
Score RMSE =  4.558935970465732e+17
Score RMSE =  1.3749540505367778e+17
Score RMSE =  1.078157664861803e+17
Score RMSE =  1.061759936897099e+17
Score RMSE =  5.272190866220759e+16
Score RMSE =  1.0527296335076926e+17
Score RMSE =  8.225055493150285e+16
Score RMSE =  2.249299268119418e+16
Score RMSE =  8.06236256405315e+16
Score RMSE =  1.1323650455987347e+17
Score RMSE =  5.316624888058047e+16
Score RMSE =  1.318947695150932e+17
Score RMSE =  1.3854001301173989e+17
Score RMSE =  2.0376796514539347e+17
Score RMSE =  1.0335417592617016e+17
Score RMSE =  1.760975658538119e+17


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None.

In [34]:
print("Score RMSE = ", rmse(target, bestmodel.predict(train)))

Score RMSE =  1.890478567832039e+17


In [36]:
test.shape

(150,)

In [0]:


submit = pd.DataFrame()
submit['Id'] = test['Id']
submit['SalePrice'] = bestmodel.predict(test)
submit.to_csv('submit.csv', index=False)